In [1]:
from Server import *
from SharePassword import *
from DigitalSignature import *
from PKAuthenticatedEnc import *
from HybridEncryption import *
from ForwardSecrecy import *

# Setup

In [2]:
server = Server()
# Bohan send message to Alice
SKB, PKB = GenKeyPair()
SKA, PKA = GenKeyPair()

preSKB, prePKB = GenPreKey(SKB)
preSKA, prePKA = GenPreKey(SKA)

boolean, tokenB = server.signup("Bohan", PKB, PKB)
boolean, tokenA = server.signup("Alice", PKA, PKB)

payload = {'api_token': tokenB,
           'to': 'Alice',
           'message': 'MTIzNA=='}
server.send(payload)
payload = {'api_token': tokenB,
           'to': 'Alice',
           'message': '1234'}

server.send(payload)
payload = {'api_token': tokenA}
server.receive(payload)

(True,
 [{'id': 0, 'from': 'Bohan', 'message': 'MTIzNA=='},
  {'id': 1, 'from': 'Bohan', 'message': '1234'}])

# 1. Pre-shared Password method

Password is secretly shared. All other parameters are in public.

In [3]:
oplimit = 524288
key_length = 32
memlimit = 16777216
salt = b"\x16\xb5\xaf\xcf\xf4\x61\xc7\x5c\x09\xb3\xe3\xe8\x16\xc9\x0d\x33\x05\xb6\x47\x53\xb3\xa2\xb0\xc5\x94\x1a\xd0\x95\xf2\x80\xbc\xfe"

cipher = EncryptSP("1234", "direct behavior", salt, oplimit, memlimit, key_length)
DecryptSP(cipher, "direct behavior", salt, oplimit, memlimit, key_length)

'1234'

# 2. Sign with Digital Signature

In [4]:
SK, PK = GenKeyPair()
signature = Sign('Hello', SK)
Verify(signature, PK)

(True, 'Hello')

# 3. Public-Key Authenticated Encryption

This will be End2End Encryption and Decryption scheme using Public Key

In [5]:
FingerPrint(PK)
SKA, PKA = GenKeyPairAE()
SKB, PKB = GenKeyPairAE()
cipher = EncryptAE('Hello', SKA, PKB)
DecryptAE(cipher, SKB, PKA)

'Hello'

# 4. Hybrid Encryption - Government Surveillance

Government will have a keyPair used for encryption and decryption

In [6]:
SK1, PK1 = GenKeyPairAE()
SK2, PK2 = GenKeyPairAE()
SKgov, PKgov = GenKeyPairAE()
cipher = EncryptHybrid("Hello", SK1, PK2, PKgov)
DecryptHybridReceive(cipher, SK2, PK1)
DecryptHybridGov(cipher, SKgov, PK1)

'Hello'

# 5. Perfect Forward Secrecy

In [7]:
SK1, PK1 = GenKeyPair()
SK2, PK2 = GenKeyPair()

preSK1, prePK1 = GenPreKey(SK1)
preSK2, prePK2 = GenPreKey(SK2)

cipher = EncryptFS('Hello', preSK1, PK2, prePK2)
DecryptFS(cipher, preSK2, PK1, prePK1)

'Hello'